In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nasa-cmaps/CMaps/RUL_FD002.txt
/kaggle/input/nasa-cmaps/CMaps/test_FD003.txt
/kaggle/input/nasa-cmaps/CMaps/Damage Propagation Modeling.pdf
/kaggle/input/nasa-cmaps/CMaps/readme.txt
/kaggle/input/nasa-cmaps/CMaps/train_FD003.txt
/kaggle/input/nasa-cmaps/CMaps/test_FD004.txt
/kaggle/input/nasa-cmaps/CMaps/train_FD004.txt
/kaggle/input/nasa-cmaps/CMaps/x.txt
/kaggle/input/nasa-cmaps/CMaps/test_FD002.txt
/kaggle/input/nasa-cmaps/CMaps/train_FD001.txt
/kaggle/input/nasa-cmaps/CMaps/train_FD002.txt
/kaggle/input/nasa-cmaps/CMaps/RUL_FD001.txt
/kaggle/input/nasa-cmaps/CMaps/RUL_FD004.txt
/kaggle/input/nasa-cmaps/CMaps/RUL_FD003.txt
/kaggle/input/nasa-cmaps/CMaps/test_FD001.txt
/kaggle/input/nasa-cmaps/cmaps/CMaps/RUL_FD002.txt
/kaggle/input/nasa-cmaps/cmaps/CMaps/test_FD003.txt
/kaggle/input/nasa-cmaps/cmaps/CMaps/Damage Propagation Modeling.pdf
/kaggle/input/nasa-cmaps/cmaps/CMaps/readme.txt
/kaggle/input/nasa-cmaps/cmaps/CMaps/train_FD003.txt
/kaggle/input/nasa-cmaps/cmaps/CM

In [2]:
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import plotly.io as pio
pio.renderers.default='kaggle'

In [3]:
# set the max columns to none
pd.set_option('display.max_columns', None)

In [4]:
train_1=pd.read_csv("/kaggle/input/nasa-cmaps/CMaps/train_FD002.txt",sep="\s+",header=None)
train_2=pd.read_csv("/kaggle/input/nasa-cmaps/CMaps/train_FD004.txt",sep="\s+",header=None)

In [5]:
# column_names=["engine", "cycle", "setting1", "setting2", "setting3", "sensor1", 
#               "sensor2", "sensor3", "sensor4", "sensor5", "sensor6", "sensor7", 
#               "sensor8", "sensor9", "sensor10", "sensor11", "sensor12", "sensor13",
#               "sensor14", "sensor15", "sensor16", "sensor17", "sensor18", "sensor19", "sensor20", "sensor21"]

index_name = ["Engine Number", "Times/ in cycle"]
setting_name = ["Altitude(Setting_1)", "Mach Number(Setting_2)", "TRA(Setting_3)"]
sensor_name = ["Total Fan inlet temperature", "Total LPC outlet temperature", "Total HPC outlet temperature", "Total LPT outlet temperature",
            "Total Fan inlet pressure","Total bypass-duct pressure", "Total HPC outlet pressure", "Physical fan speed", "Physical core speed","Engine pressure ratio(P50/P2)", "Static HPC outlet pressure",
            "Ratio of fuel flow to Ps30", "Corrected fan speed", "Corrected core speed", "Bypass Ratio", "Burner fuel-air ratio", "Bleed enthalpy", "Required fan speed", "Required fan conversion speed",
            "HPT coolant bleed", "LPT coolant bleed"]

column_names=index_name+setting_name+sensor_name
column_names

['Engine Number',
 'Times/ in cycle',
 'Altitude(Setting_1)',
 'Mach Number(Setting_2)',
 'TRA(Setting_3)',
 'Total Fan inlet temperature',
 'Total LPC outlet temperature',
 'Total HPC outlet temperature',
 'Total LPT outlet temperature',
 'Total Fan inlet pressure',
 'Total bypass-duct pressure',
 'Total HPC outlet pressure',
 'Physical fan speed',
 'Physical core speed',
 'Engine pressure ratio(P50/P2)',
 'Static HPC outlet pressure',
 'Ratio of fuel flow to Ps30',
 'Corrected fan speed',
 'Corrected core speed',
 'Bypass Ratio',
 'Burner fuel-air ratio',
 'Bleed enthalpy',
 'Required fan speed',
 'Required fan conversion speed',
 'HPT coolant bleed',
 'LPT\xa0coolant\xa0bleed']

In [6]:
train_1.columns=column_names
train_2.columns=column_names

In [7]:
display(train_1.head())

,Engine Number,Times/ in cycle,Altitude(Setting_1),Mach Number(Setting_2),TRA(Setting_3),Total Fan inlet temperature,Total LPC outlet temperature,Total HPC outlet temperature,Total LPT outlet temperature,Total Fan inlet pressure,Total bypass-duct pressure,Total HPC outlet pressure,Physical fan speed,Physical core speed,Engine pressure ratio(P50/P2),Static HPC outlet pressure,Ratio of fuel flow to Ps30,Corrected fan speed,Corrected core speed,Bypass Ratio,Burner fuel-air ratio,Bleed enthalpy,Required fan speed,Required fan conversion speed,HPT coolant bleed,LPT coolant bleed
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,2222.65,8341.91,1.02,42.02,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,2211.57,8303.96,1.02,42.20,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,1915.11,8001.42,0.94,36.69,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,2211.58,8303.96,1.02,41.96,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,1915.10,7993.23,0.94,36.89,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


In [8]:
display(train_2.head())

,Engine Number,Times/ in cycle,Altitude(Setting_1),Mach Number(Setting_2),TRA(Setting_3),Total Fan inlet temperature,Total LPC outlet temperature,Total HPC outlet temperature,Total LPT outlet temperature,Total Fan inlet pressure,Total bypass-duct pressure,Total HPC outlet pressure,Physical fan speed,Physical core speed,Engine pressure ratio(P50/P2),Static HPC outlet pressure,Ratio of fuel flow to Ps30,Corrected fan speed,Corrected core speed,Bypass Ratio,Burner fuel-air ratio,Bleed enthalpy,Required fan speed,Required fan conversion speed,HPT coolant bleed,LPT coolant bleed
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,137.98,2211.88,8312.35,1.02,41.68,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,9.00,174.82,1915.22,7994.94,0.93,36.48,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


In [9]:
train_1.shape,train_2.shape

((53759, 26), (61249, 26))

In [10]:
display(train_1.describe().T)
display(train_2.describe().T)

,count,mean,std,min,25%,50%,75%,max
Engine Number,53759.0,131.082981,74.463862,1.0000,68.0000,131.0000,195.0000,260.0000
Times/ in cycle,53759.0,109.154746,69.180569,1.0000,52.0000,104.0000,157.0000,378.0000
Altitude(Setting_1),53759.0,23.998407,14.747376,0.0000,10.0046,25.0013,41.9980,42.0080
Mach Number(Setting_2),53759.0,0.572056,0.310016,0.0000,0.2507,0.7000,0.8400,0.8420
TRA(Setting_3),53759.0,94.046020,14.237735,60.0000,100.0000,100.0000,100.0000,100.0000
Total Fan inlet temperature,53759.0,472.910207,26.389707,445.0000,445.0000,462.5400,491.1900,518.6700
Total LPC outlet temperature,53759.0,579.672399,37.289399,535.5300,549.5700,555.9800,607.3400,644.5200
Total HPC outlet temperature,53759.0,1419.971013,105.946341,1243.7300,1352.7600,1369.1800,1499.3700,1612.8800
Total LPT outlet temperature,53759.0,1205.442024,119.123428,1023.7700,1123.6550,1138.8900,1306.8500,1439.2300
Total Fan inlet pressure,53759.0,8.031986,3.613839,3.9100,3.9100,7.0500,10.5200,14.6200


,count,mean,std,min,25%,50%,75%,max
Engine Number,61249.0,124.325181,71.995350,1.0000,60.0000,126.0000,185.0000,249.0000
Times/ in cycle,61249.0,134.311417,89.783389,1.0000,62.0000,123.0000,191.0000,543.0000
Altitude(Setting_1),61249.0,23.999823,14.780722,0.0000,10.0046,25.0014,41.9981,42.0080
Mach Number(Setting_2),61249.0,0.571347,0.310703,0.0000,0.2507,0.7000,0.8400,0.8420
TRA(Setting_3),61249.0,94.031576,14.251954,60.0000,100.0000,100.0000,100.0000,100.0000
Total Fan inlet temperature,61249.0,472.882435,26.436832,445.0000,445.0000,462.5400,491.1900,518.6700
Total LPC outlet temperature,61249.0,579.420056,37.342647,535.4800,549.3300,555.7400,607.0700,644.4200
Total HPC outlet temperature,61249.0,1417.896600,106.167598,1242.6700,1350.5500,1367.6800,1497.4200,1613.0000
Total LPT outlet temperature,61249.0,1201.915359,119.327591,1024.4200,1119.4900,1136.9200,1302.6200,1440.7700
Total Fan inlet pressure,61249.0,8.031626,3.622872,3.9100,3.9100,7.0500,10.5200,14.6200


In [11]:
display(train_1.info())
display(train_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53759 entries, 0 to 53758
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Engine Number                  53759 non-null  int64  
 1   Times/ in cycle                53759 non-null  int64  
 2   Altitude(Setting_1)            53759 non-null  float64
 3   Mach Number(Setting_2)         53759 non-null  float64
 4   TRA(Setting_3)                 53759 non-null  float64
 5   Total Fan inlet temperature    53759 non-null  float64
 6   Total LPC outlet temperature   53759 non-null  float64
 7   Total HPC outlet temperature   53759 non-null  float64
 8   Total LPT outlet temperature   53759 non-null  float64
 9   Total Fan inlet pressure       53759 non-null  float64
 10  Total bypass-duct pressure     53759 non-null  float64
 11  Total HPC outlet pressure      53759 non-null  float64
 12  Physical fan speed             53759 non-null 

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61249 entries, 0 to 61248
Data columns (total 26 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Engine Number                  61249 non-null  int64  
 1   Times/ in cycle                61249 non-null  int64  
 2   Altitude(Setting_1)            61249 non-null  float64
 3   Mach Number(Setting_2)         61249 non-null  float64
 4   TRA(Setting_3)                 61249 non-null  float64
 5   Total Fan inlet temperature    61249 non-null  float64
 6   Total LPC outlet temperature   61249 non-null  float64
 7   Total HPC outlet temperature   61249 non-null  float64
 8   Total LPT outlet temperature   61249 non-null  float64
 9   Total Fan inlet pressure       61249 non-null  float64
 10  Total bypass-duct pressure     61249 non-null  float64
 11  Total HPC outlet pressure      61249 non-null  float64
 12  Physical fan speed             61249 non-null 

None

In [12]:
print(train_1.isnull().sum())
print(train_2.isnull().sum())

Engine Number                    0
Times/ in cycle                  0
Altitude(Setting_1)              0
Mach Number(Setting_2)           0
TRA(Setting_3)                   0
Total Fan inlet temperature      0
Total LPC outlet temperature     0
Total HPC outlet temperature     0
Total LPT outlet temperature     0
Total Fan inlet pressure         0
Total bypass-duct pressure       0
Total HPC outlet pressure        0
Physical fan speed               0
Physical core speed              0
Engine pressure ratio(P50/P2)    0
Static HPC outlet pressure       0
Ratio of fuel flow to Ps30       0
Corrected fan speed              0
Corrected core speed             0
Bypass Ratio                     0
Burner fuel-air ratio            0
Bleed enthalpy                   0
Required fan speed               0
Required fan conversion speed    0
HPT coolant bleed                0
LPT coolant bleed                0
dtype: int64
Engine Number                    0
Times/ in cycle                  0
Altitud

In [13]:
print((train_1.duplicated()==1).sum())
print((train_2.duplicated()==1).sum())

0
0


In [14]:
display(train_1.head())
display(train_2.head())

,Engine Number,Times/ in cycle,Altitude(Setting_1),Mach Number(Setting_2),TRA(Setting_3),Total Fan inlet temperature,Total LPC outlet temperature,Total HPC outlet temperature,Total LPT outlet temperature,Total Fan inlet pressure,Total bypass-duct pressure,Total HPC outlet pressure,Physical fan speed,Physical core speed,Engine pressure ratio(P50/P2),Static HPC outlet pressure,Ratio of fuel flow to Ps30,Corrected fan speed,Corrected core speed,Bypass Ratio,Burner fuel-air ratio,Bleed enthalpy,Required fan speed,Required fan conversion speed,HPT coolant bleed,LPT coolant bleed
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,2222.65,8341.91,1.02,42.02,183.06,2387.72,8048.56,9.3461,0.02,334,2223,100.00,14.73,8.8071
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,2211.57,8303.96,1.02,42.20,130.42,2387.66,8072.30,9.3774,0.02,330,2212,100.00,10.41,6.2665
2,1,3,24.9988,0.6218,60.0,462.54,537.31,1256.76,1047.45,7.05,9.02,175.71,1915.11,8001.42,0.94,36.69,164.22,2028.03,7864.87,10.8941,0.02,309,1915,84.93,14.08,8.6723
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,2211.58,8303.96,1.02,41.96,130.72,2387.61,8068.66,9.3528,0.02,329,2212,100.00,10.59,6.4701
4,1,5,25.0005,0.6203,60.0,462.54,537.07,1257.71,1047.93,7.05,9.03,175.05,1915.10,7993.23,0.94,36.89,164.31,2028.00,7861.23,10.8963,0.02,309,1915,84.93,14.13,8.5286


,Engine Number,Times/ in cycle,Altitude(Setting_1),Mach Number(Setting_2),TRA(Setting_3),Total Fan inlet temperature,Total LPC outlet temperature,Total HPC outlet temperature,Total LPT outlet temperature,Total Fan inlet pressure,Total bypass-duct pressure,Total HPC outlet pressure,Physical fan speed,Physical core speed,Engine pressure ratio(P50/P2),Static HPC outlet pressure,Ratio of fuel flow to Ps30,Corrected fan speed,Corrected core speed,Bypass Ratio,Burner fuel-air ratio,Bleed enthalpy,Required fan speed,Required fan conversion speed,HPT coolant bleed,LPT coolant bleed
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330,2212,100.00,10.62,6.3670
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361,2324,100.00,24.37,14.6552
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329,2212,100.00,10.48,6.4213
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,137.98,2211.88,8312.35,1.02,41.68,129.80,2388.02,8076.05,9.3369,0.02,328,2212,100.00,10.54,6.4176
4,1,5,25.0063,0.6207,60.0,462.54,536.10,1255.23,1033.59,7.05,9.00,174.82,1915.22,7994.94,0.93,36.48,164.11,2028.08,7865.80,10.8366,0.02,305,1915,84.93,14.03,8.6754


#### Features
"engine" : Engine No.

"cycle" : Time, In Cycles #### 2. Setting Names:

"setting1" : Operation Setting 1

"setting2" : Operation Setting 2

"setting3" : Operation Setting 3 #### 3. Sensor Names:

"sensor1" : Fan Inlet Temperature (◦R)

"sensor2" : LPC Outlet Temperature (◦R)

"sensor3" : HPC Outlet Temperature (◦R)

"sensor4" : LPT Outlet Temperature (◦R)

"sensor5" : Fan Inlet Pressure (psia)

"sensor6" : Bypass-Duct Pressure (psia)

"sensor7" : HPC Outlet Pressure (psia)

"sensor8" : Physical Fan Speed (rpm)

"sensor9" : Physical Core Speed (rpm)

"sensor10" : Engine Pressure Ratio(P50/P2)

"sensor11" : HPC Outlet Static Pressure (psia)

"sensor12" : Ratio of Fuel Flow to Ps30 (pps/psia)

"sensor13" : Corrected Fan Speed (rpm)

"sensor14" : Corrected Core Speed (rpm)

"sensor15" : Bypass Ratio

"sensor16" : Burner Fuel-Air Ratio

"sensor17" : Bleed Enthalpy

"sensor18" : Required Fan Speed

"sensor19" : Required Fan Conversion Speed

"sensor20" : High-Pressure Turbines Cool Air Flow

"sensor21" : Low-Pressure Turbines Cool Air Flow

In [15]:
display(train_1.nunique())
print("\n")
display(train_2.nunique())

Engine Number                      260
Times/ in cycle                    378
Altitude(Setting_1)                536
Mach Number(Setting_2)             105
TRA(Setting_3)                       2
Total Fan inlet temperature          6
Total LPC outlet temperature      1590
Total HPC outlet temperature     12305
Total LPT outlet temperature     15411
Total Fan inlet pressure             6
Total bypass-duct pressure          14
Total HPC outlet pressure         2067
Physical fan speed                 897
Physical core speed              22434
Engine pressure ratio(P50/P2)        9
Static HPC outlet pressure         681
Ratio of fuel flow to Ps30        1672
Corrected fan speed                514
Corrected core speed             14905
Bypass Ratio                      8464
Burner fuel-air ratio                2
Bleed enthalpy                      53
Required fan speed                   6
Required fan conversion speed        2
HPT coolant bleed                  510
LPT coolant bleed        

Engine Number                      249
Times/ in cycle                    543
Altitude(Setting_1)                536
Mach Number(Setting_2)             105
TRA(Setting_3)                       2
Total Fan inlet temperature          6
Total LPC outlet temperature      1704
Total HPC outlet temperature     13558
Total LPT outlet temperature     17353
Total Fan inlet pressure             6
Total bypass-duct pressure          46
Total HPC outlet pressure         5926
Physical fan speed                1038
Physical core speed              25297
Engine pressure ratio(P50/P2)       21
Static HPC outlet pressure         737
Ratio of fuel flow to Ps30        5627
Corrected fan speed                483
Corrected core speed             15938
Bypass Ratio                     11915
Burner fuel-air ratio                2
Bleed enthalpy                      54
Required fan speed                   6
Required fan conversion speed        2
HPT coolant bleed                  652
LPT coolant bleed        

In [16]:
display(train_1[["Engine Number","Times/ in cycle"]].groupby("Times/ in cycle").max())
display(train_2[["Engine Number","Times/ in cycle"]].groupby("Times/ in cycle").max())

,Engine Number
Times/ in cycle,
1,260
2,260
3,260
4,260
5,260
...,...
374,112
375,112
376,112


,Engine Number
Times/ in cycle,
1,249
2,249
3,249
4,249
5,249
...,...
539,118
540,118
541,118


In [17]:
print(train_1["Times/ in cycle"].max())
print(print(train_2["Times/ in cycle"].max()))

378
543
None


In [18]:
fig = px.bar(x=train_1["Engine Number"], y=train_1["Times/ in cycle"])
fig.show()

In [19]:
fig = px.bar(x=train_2["Engine Number"], y=train_2["Times/ in cycle"])
fig.show()

In [20]:
df1=train_1.groupby('Engine Number')[['Times/ in cycle']].max()
df1[df1['Times/ in cycle'] == df1['Times/ in cycle'].max()]

,Times/ in cycle
Engine Number,
112,378


In [21]:
df2=train_2.groupby('Engine Number')[['Times/ in cycle']].max()
df2[df2['Times/ in cycle'] == df2['Times/ in cycle'].max()]

,Times/ in cycle
Engine Number,
118,543


In [22]:
def Calculate_RUL(df):
    max_cycles = df.groupby('Engine Number')['Times/ in cycle'].max()
    merged = df.merge(max_cycles.to_frame(name='max_time_cycle'), left_on='Engine Number',right_index=True)
    merged["RUL"] = merged["max_time_cycle"] - merged['Times/ in cycle']
    merged = merged.drop("max_time_cycle", axis=1)
    return merged

In [23]:
train_1=Calculate_RUL(train_1)
train_2=Calculate_RUL(train_2)

In [24]:
print(train_1.columns)
print(train_2.columns)

Index(['Engine Number', 'Times/ in cycle', 'Altitude(Setting_1)',
       'Mach Number(Setting_2)', 'TRA(Setting_3)',
       'Total Fan inlet temperature', 'Total LPC outlet temperature',
       'Total HPC outlet temperature', 'Total LPT outlet temperature',
       'Total Fan inlet pressure', 'Total bypass-duct pressure',
       'Total HPC outlet pressure', 'Physical fan speed',
       'Physical core speed', 'Engine pressure ratio(P50/P2)',
       'Static HPC outlet pressure', 'Ratio of fuel flow to Ps30',
       'Corrected fan speed', 'Corrected core speed', 'Bypass Ratio',
       'Burner fuel-air ratio', 'Bleed enthalpy', 'Required fan speed',
       'Required fan conversion speed', 'HPT coolant bleed',
       'LPT coolant bleed', 'RUL'],
      dtype='object')
Index(['Engine Number', 'Times/ in cycle', 'Altitude(Setting_1)',
       'Mach Number(Setting_2)', 'TRA(Setting_3)',
       'Total Fan inlet temperature', 'Total LPC outlet temperature',
       'Total HPC outlet temperature', 'T

In [ ]:
display(train_1.corr())
display(train_2.corr())

In [ ]:
sns.heatmap(train_1.corr(),annot=True,linewidths=0.2)
fig=plt.gcf()
fig.set_size_inches(20,20)
plt.show()

In [ ]:
sns.heatmap(train_2.corr(),annot=True,linewidths=0.2)
fig=plt.gcf()
fig.set_size_inches(20,20)
plt.show()

In [ ]:
def box_plot(df,col):
    fig=px.box(df,x=col)
    fig.show()

In [ ]:
sensors_1=train_1.drop(columns=['Engine Number', 'Times/ in cycle', 'RUL'],axis=1)
sensors_1=sensors_1.columns

for sensor_1 in sensors_1:
    box_plot(train_1,sensor_1)

In [ ]:
sensors_2=train_2.drop(columns=['Engine Number', 'Times/ in cycle', 'RUL'],axis=1)
sensors_2=sensors_2.columns

for sensor_2 in sensors_2:
    box_plot(train_2,sensor_2)

In [ ]:
def remove_outliers(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    col_filtered = col[(col > lower_bound) & (col < upper_bound)]
    return col_filtered

In [ ]:
def impute_median(col):
    col = col.copy() 
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    
    col[col < lower_bound] = col.median() 
    col[col > upper_bound] = col.median()
    
    return col



In [25]:
# columns_with_outlier_train_1=["Physical fan speed","Static HPC outlet pressure","Corrected fan speed","Corrected core speed","Bypass Ratio","Required fan speed","Required fan conversion speed"]
sensors_1=train_1.drop(columns=['Engine Number', 'Times/ in cycle', 'RUL'],axis=1)
sensors_1=sensors_1.columns
for col in sensors_1:
    q1 = train_1[col].quantile(0.25)
    q3 = train_1[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    train_1.loc[train_1[col] < lower_bound, col] = np.nan
    train_1.loc[train_1[col] > upper_bound, col] = np.nan

In [26]:
train_1=train_1.dropna()

In [27]:
train_1.shape

(44067, 27)

In [28]:
train_1.isnull().sum()

Engine Number                    0
Times/ in cycle                  0
Altitude(Setting_1)              0
Mach Number(Setting_2)           0
TRA(Setting_3)                   0
Total Fan inlet temperature      0
Total LPC outlet temperature     0
Total HPC outlet temperature     0
Total LPT outlet temperature     0
Total Fan inlet pressure         0
Total bypass-duct pressure       0
Total HPC outlet pressure        0
Physical fan speed               0
Physical core speed              0
Engine pressure ratio(P50/P2)    0
Static HPC outlet pressure       0
Ratio of fuel flow to Ps30       0
Corrected fan speed              0
Corrected core speed             0
Bypass Ratio                     0
Burner fuel-air ratio            0
Bleed enthalpy                   0
Required fan speed               0
Required fan conversion speed    0
HPT coolant bleed                0
LPT coolant bleed                0
RUL                              0
dtype: int64

In [29]:
train_1.head()

,Engine Number,Times/ in cycle,Altitude(Setting_1),Mach Number(Setting_2),TRA(Setting_3),Total Fan inlet temperature,Total LPC outlet temperature,Total HPC outlet temperature,Total LPT outlet temperature,Total Fan inlet pressure,Total bypass-duct pressure,Total HPC outlet pressure,Physical fan speed,Physical core speed,Engine pressure ratio(P50/P2),Static HPC outlet pressure,Ratio of fuel flow to Ps30,Corrected fan speed,Corrected core speed,Bypass Ratio,Burner fuel-air ratio,Bleed enthalpy,Required fan speed,Required fan conversion speed,HPT coolant bleed,LPT coolant bleed,RUL
0,1,1,34.9983,0.8400,100.0,449.44,555.32,1358.61,1137.23,5.48,8.00,194.64,2222.65,8341.91,1.02,42.02,183.06,2387.72,8048.56,9.3461,0.02,334.0,2223.0,100.0,14.73,8.8071,148
1,1,2,41.9982,0.8408,100.0,445.00,549.90,1353.22,1125.78,3.91,5.71,138.51,2211.57,8303.96,1.02,42.20,130.42,2387.66,8072.30,9.3774,0.02,330.0,2212.0,100.0,10.41,6.2665,147
3,1,4,42.0077,0.8416,100.0,445.00,549.51,1354.03,1126.38,3.91,5.71,138.46,2211.58,8303.96,1.02,41.96,130.72,2387.61,8068.66,9.3528,0.02,329.0,2212.0,100.0,10.59,6.4701,145
6,1,7,42.0043,0.8409,100.0,445.00,549.74,1347.45,1127.19,3.91,5.71,138.71,2211.62,8307.75,1.02,42.19,130.49,2387.69,8075.54,9.3753,0.02,330.0,2212.0,100.0,10.62,6.4227,142
7,1,8,20.0020,0.7002,100.0,491.19,607.44,1481.69,1252.36,9.35,13.65,334.41,2323.87,8709.12,1.08,44.27,315.11,2387.99,8049.26,9.2369,0.02,365.0,2324.0,100.0,24.33,14.7989,141


In [ ]:
# columns_imputed_train_1=["setting3","sensor8","sensor11","sensor13","sensor14","sensor15","sensor19"]

# for col in columns_imputed_train_1:
#     box_plot(train_1,col)

In [30]:
# columns_with_outlier_train_2=["sensor8","sensor11","sensor13","sensor14","sensor15"]
sensors_2=train_2.drop(columns=['Engine Number', 'Times/ in cycle', 'RUL'],axis=1)
sensors_2=sensors_2.columns
for col in sensors_2:
    q1 = train_2[col].quantile(0.25)
    q3 = train_2[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)

    train_2.loc[train_2[col] < lower_bound, col] = np.nan
    train_2.loc[train_2[col] > upper_bound, col] = np.nan

In [31]:
train_2=train_2.dropna()

In [32]:
train_2.head()

,Engine Number,Times/ in cycle,Altitude(Setting_1),Mach Number(Setting_2),TRA(Setting_3),Total Fan inlet temperature,Total LPC outlet temperature,Total HPC outlet temperature,Total LPT outlet temperature,Total Fan inlet pressure,Total bypass-duct pressure,Total HPC outlet pressure,Physical fan speed,Physical core speed,Engine pressure ratio(P50/P2),Static HPC outlet pressure,Ratio of fuel flow to Ps30,Corrected fan speed,Corrected core speed,Bypass Ratio,Burner fuel-air ratio,Bleed enthalpy,Required fan speed,Required fan conversion speed,HPT coolant bleed,LPT coolant bleed,RUL
0,1,1,42.0049,0.8400,100.0,445.00,549.68,1343.43,1112.93,3.91,5.70,137.36,2211.86,8311.32,1.01,41.69,129.78,2387.99,8074.83,9.3335,0.02,330.0,2212.0,100.0,10.62,6.3670,320
1,1,2,20.0020,0.7002,100.0,491.19,606.07,1477.61,1237.50,9.35,13.61,332.10,2323.66,8713.60,1.07,43.94,312.59,2387.73,8046.13,9.1913,0.02,361.0,2324.0,100.0,24.37,14.6552,319
2,1,3,42.0038,0.8409,100.0,445.00,548.95,1343.12,1117.05,3.91,5.69,138.18,2211.92,8306.69,1.01,41.66,129.62,2387.97,8066.62,9.4007,0.02,329.0,2212.0,100.0,10.48,6.4213,318
3,1,4,42.0000,0.8400,100.0,445.00,548.70,1341.24,1118.03,3.91,5.70,137.98,2211.88,8312.35,1.02,41.68,129.80,2388.02,8076.05,9.3369,0.02,328.0,2212.0,100.0,10.54,6.4176,317
5,1,6,34.9996,0.8400,100.0,449.44,554.77,1352.87,1117.01,5.48,7.97,193.82,2222.77,8340.00,1.02,41.44,181.90,2387.87,8054.10,9.3346,0.02,330.0,2223.0,100.0,14.91,8.9057,315


In [33]:
# columns_imputed_train_2=["setting3","sensor13","sensor14","sensor15","sensor19"]


# for col in columns_imputed_train_2:
#     box_plot(train_2,col)

In [34]:
from sklearn.model_selection import train_test_split

In [ ]:
# # # X1.isnull().sum()
# index_name = ["Engine Number", "Times/ in cycle"]
# setting_name = ["Altitude(Setting_1)", "Mach Number(Setting_2)", "TRA(Setting_3)"]
# sensor_name = ["Total Fan inlet temperature", "Total LPC outlet temperature", "Total HPC outlet temperature", "Total LPT outlet temperature",
#             "Total Fan inlet pressure","Total bypass-duct pressure", "Total HPC outlet pressure", "Physical fan speed", "Physical core speed","Engine pressure ratio(P50/P2)", "Static HPC outlet pressure",
#             "Ratio of fuel flow to Ps30", "Corrected fan speed", "Corrected core speed", "Bypass Ratio", "Burner fuel-air ratio", "Bleed enthalpy", "Required fan speed", "Required fan conversion speed",
#             "HPT coolant bleed", "LPT coolant bleed"

In [35]:
X1=train_1.drop(["Engine Number","Times/ in cycle","Burner fuel-air ratio","Required fan speed","Total Fan inlet temperature",
                 "Total Fan inlet pressure","Required fan conversion speed","Mach Number(Setting_2)","TRA(Setting_3)",
                 "Bleed enthalpy","Engine pressure ratio(P50/P2)","RUL"],axis=1)
y1=train_1["RUL"]
print(X1.shape)
# print(X1)
X1_train,X1_valid,y1_train,y1_valid=train_test_split(X1,y1,test_size=0.2,random_state=29)

(44067, 15)


In [36]:
X2=train_2.drop(["Engine Number","Times/ in cycle","Burner fuel-air ratio","Required fan speed","Total Fan inlet temperature",
                 "Total Fan inlet pressure","Bleed enthalpy","Engine pressure ratio(P50/P2)",
                 "Required fan conversion speed", "Mach Number(Setting_2)","TRA(Setting_3)","RUL"],axis=1)
y2=train_2["RUL"]

X2_train,X2_valid,y2_train,y2_valid=train_test_split(X2,y2,test_size=0.2,random_state=29)

In [37]:
rul_1=pd.read_csv("/kaggle/input/nasa-cmaps/CMaps/RUL_FD002.txt",header=None, names=["RUL"])
rul_2=pd.read_csv("/kaggle/input/nasa-cmaps/CMaps/RUL_FD004.txt",header=None, names=["RUL"])

In [38]:
test_1=pd.read_csv('/kaggle/input/nasa-cmaps/CMaps/test_FD002.txt',sep="\s+",header=None)
test_2=pd.read_csv('/kaggle/input/nasa-cmaps/CMaps/test_FD004.txt',sep="\s+",header=None)

In [39]:
test_1.columns=column_names
test_2.columns=column_names

In [ ]:
from sklearn.preprocessing import StandardScaler

In [40]:
# scaler = MinMaxScaler()
scaler=StandardScaler()

In [47]:
## Scaling X_train1 and X_test1

scaled_train_1=scaler.fit_transform(X1_train)
scaled_valid_1=scaler.fit_transform(X1_valid)

## Conserve only the last occurence of each unit to match the length of test1

X_test_1 = test_1.groupby('Engine Number').last().reset_index().drop(columns=["Engine Number","Times/ in cycle","Burner fuel-air ratio",
                                                                              "Required fan speed","Total Fan inlet temperature","Total Fan inlet pressure",
                                                                              "Required fan conversion speed", "Mach Number(Setting_2)",
                                                                              "Bleed enthalpy","Engine pressure ratio(P50/P2)","TRA(Setting_3)"],axis=1)

## scaling X_valid

# scaled_X_test_1=scaler.fit_transform(X_test_1)

In [48]:
print(f"X_train1: {X1_train.shape}")
print(f"X1_valid: {X1_valid.shape}")
print(f"y1_train: {y1_train.shape}")
print(f"y1_valid: {y1_valid.shape}")
print(f"X_test_1: {X_test_1.shape}")
print(f"rul_1: {rul_1.shape}")

print(f"scaled_train_1: {scaled_train_1.shape}")
print(f"scaled_valid_1: {scaled_valid_1.shape}")
print(f"scaled_X_test1: {X_test_1.shape}")

X_train1: (35253, 15)
X1_valid: (8814, 15)
y1_train: (35253,)
y1_valid: (8814,)
X_test_1: (259, 15)
rul_1: (259, 1)
scaled_train_1: (35253, 15)
scaled_valid_1: (8814, 15)
scaled_X_test1: (259, 15)


In [49]:
scaled_train_2=scaler.fit_transform(X2_train)
scaled_valid_2=scaler.fit_transform(X2_valid)

## Conserve only the last occurence of each unit to match the length of test1

X_test_2 = test_2.groupby('Engine Number').last().reset_index().drop(columns=["Engine Number","Times/ in cycle","Burner fuel-air ratio",
                                                                              "Required fan speed","Total Fan inlet temperature","Total Fan inlet pressure",
                                                                              "Required fan conversion speed", "Mach Number(Setting_2)",
                                                                              "Bleed enthalpy","Engine pressure ratio(P50/P2)","TRA(Setting_3)"],axis=1)

# scaled_X_test_2=scaler.fit_transform(X_test_2)

In [50]:
print(f"X2_train: {X2_train.shape}")
print(f"X2_valid: {X2_valid.shape}")
print(f"y2_train: {y2_train.shape}")
print(f"y2_valid: {y2_valid.shape}")
print(f"X_test_2: {X_test_2.shape}")
print(f"rul_2: {rul_2.shape}")

print(f"scaled_train_2: {scaled_train_2.shape}")
print(f"scaled_valid_2: {scaled_valid_2.shape}")
print(f"scaled_X_test2: {X_test_2.shape}")

X2_train: (39744, 15)
X2_valid: (9936, 15)
y2_train: (39744,)
y2_valid: (9936,)
X_test_2: (248, 15)
rul_2: (248, 1)
scaled_train_2: (39744, 15)
scaled_valid_2: (9936, 15)
scaled_X_test2: (248, 15)


In [45]:
X_train=pd.concat([X1_train,X2_train])
X_valid=pd.concat([X1_valid,X2_valid])
y_train=pd.concat([y1_train,y2_train])
y_valid=pd.concat([y1_valid,y2_valid])

X_train.shape,X_valid.shape,y_train.shape,y_valid.shape

((74997, 15), (18750, 15), (74997,), (18750,))

In [52]:
X_test=pd.concat([X_test_1,X_test_2])
RUL=pd.concat([rul_1,rul_2])
X_test=scaler.fit_transform(X_test)
X_test.shape,RUL.shape

((507, 15), (507, 1))

In [53]:
X_train=scaler.fit_transform(X_train)
X_valid=scaler.fit_transform(X_valid)

In [ ]:
X_train[0]

In [ ]:
X_train.dropna(inplace=True)

In [ ]:
train_df=pd.DataFrame(X_train).isnull().sum()
train_df

In [ ]:
pd.DataFrame(X_train).isnull().sum().columns

In [54]:
def prediction( model, scaled_test, y_test):
    
    # Make predictions
    y_pred = model.predict(scaled_test)
    
    # Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred) 
    print("MSE:", mse)
    # MSE measures the average squared difference between actual and predicted values. Lower values are better.
    
    # Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print("RMSE:", rmse)
    # RMSE is the square root of MSE. Measures predictive accuracy in units of the variable. Lower is better.
    
    # R-squared (R2) 
    r2 = r2_score(y_test, y_pred)
    print("R-squared:", r2)
    # R2 measures proportion of variance in actual data explained by the prediction. Values range 0 to 1. Higher is better. 
    
    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(y_test, y_pred)  
    print("MAE:", mae)
    # MAE measures average absolute difference between actual and predicted values. Lower values are better.

    return y_pred

### Model Training

In [55]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf1=RandomForestRegressor()
rf1.fit(X_train,y_train)

In [ ]:
# params={
#     "n_estimators":[100,120,150],
#     "criterion":["squared_error","friedman_mse"],
#     "max_features":["sqrt","log2"],
#     "oob_score":[True,False]
# }

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

rf = HistGradientBoostingRegressor()
rf.fit(X_train, y_train)

In [64]:
rf1=RandomForestRegressor()
rf1.fit(X_train,y_train)

RandomForestRegressor()

In [65]:
print(f"\nRandom Forest-Regressor on merge data-test dataset:")
prediction(rf1, scaled_X_test_1, rul_1)

print(f"\nRandom Forest-Regressor on merge data-validation dataset:")
prediction(rf1, X_valid,y_valid)


Random Forest-Regressor on merge data-test dataset:
MSE: 5431.09532934363
RMSE: 73.6959654889169
R-squared: -0.877869054169361
MAE: 60.07023166023166

Random Forest-Regressor on merge data-validation dataset:
MSE: 3124.717231168
RMSE: 55.899170218957636
R-squared: 0.5143545039572728
MAE: 42.04126506666667


array([ 97.69, 170.34, 207.61, ..., 128.02, 158.91,  65.12])

In [68]:
global_importances = pd.Series(rf1.feature_importances_, index=X1.columns)
global_importances.sort_values(ascending=True, inplace=True)

df = pd.DataFrame({'importance': global_importances.values,
                   'feature': global_importances.index})

fig = px.bar(df, x='importance', y='feature', 
             orientation='h')

fig.update_layout(
    title='Global Feature Importance - Built-in Method',
    xaxis_title='Importance', 
    yaxis_title='Feature'
)

fig.show()

In [57]:
from sklearn.svm import SVR

In [58]:
svr1 = SVR()
svr1.fit(X_train, y_train)

SVR()

In [60]:
print(f"\nSupport-Vector-Regressor on FD002-test dataset:\n")
prediction(svr1, scaled_X_test_1, rul_1)

print(f"Support-Vector-Regressor on FD002-validation dataset:\n")
prediction(svr1, scaled_valid_1, y1_valid)


Support-Vector-Regressor on FD002-test dataset:

MSE: 3874.5537031139547
RMSE: 62.245913143867966
R-squared: -0.3396753429246693
MAE: 51.32808043423647
Support-Vector-Regressor on FD002-validation dataset:

MSE: 2395.128100008741
RMSE: 48.94004597473056
R-squared: 0.488076590257961
MAE: 37.19757108934698


array([ 99.15629093, 148.07325626, 177.62845676, ..., 118.10160421,
        77.34869299,  97.93617903])

In [69]:
def get_best_params(X_train,y_train,estimator,params,verbose=3,cv=5):
    cv=GridSearchCV(estimator=estimator,param_grid=params,verbose=verbose,cv=cv)
    cv.fit(X_train,y_train)
    
    return cv.best_params_

In [61]:
params={
    "kernel":["poly","sigmoid","rbf"],
    "C":[1,0.5],
    "gamma":["scale","auto"],
    "epsilon":[0.1,0.01,0.5]
}

In [62]:
cv=GridSearchCV(estimator=svr1,param_grid=params,verbose=3,cv=5)
cv.fit(scaled_train_1, y1_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5] END C=1, epsilon=0.1, gamma=scale, kernel=poly;, score=0.293 total time=  53.1s


KeyboardInterrupt: 

In [ ]:
cv.best_params_

In [ ]:
svr_hp=SVR(kernel="poly",C=1)
svr_hp.fit(scaled_train_1, y1_train)

In [ ]:
print(f"\nSupport-Vector-Regressor on FD002-test dataset:\n")
prediction(svr, scaled_X_test_1, rul_1)

print(f"Support-Vector-Regressor on FD002-validation dataset:\n")
prediction(svr, scaled_valid_1, y1_valid)